In [ ]:
# 단계 1: 폰트 설치
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

In [ ]:
# # 단계 2: 런타임 재시작
# import os
# os.kill(os.getpid(), 9)

In [ ]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False
	
# 한글 폰트 설정
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_name = fm.FontProperties(fname=path, size=18).get_name()
plt.rc('font', family=font_name)
fm._rebuild()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


한글관련 설치

In [ ]:
# !pip install sentencepiece
# !pip install python-Levenshtein
# !pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 7.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.1 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149858 sha256=5b212a76e11df713ede2e1e0969a4023d3fd4cf3b0b80b3a67f619a09c3a2f8e
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 토큰화 진행

# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install git+https://github.com/ssut/py-hanspell.git
# !pip install soynlp
# !pip install customized_konlpy

In [ ]:
# ! pip uninstall numpy
# ! pip install numpy

Found existing installation: numpy 1.19.5
Uninstalling numpy-1.19.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.19.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.19.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 7.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

## **전처리**

모듈 불러오기

In [ ]:
from collections import Counter
import pandas as pd
import gensim
from konlpy.tag import Komoran
import sentencepiece as spm
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

쏘카 메모 데이터 불러오기

In [ ]:
data_path = '/content/drive/MyDrive/AIFFELTHON/아이펠톤_도전데이터/정비메모분석/memo_hackathon_lesion_info_table.tsv'

In [ ]:
all_memo_data = pd.read_csv(data_path)
# all_memo_data.head()

우선 이름 변경

In [ ]:
all_memo_data.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
# # all_memo_data.head()

memo와, inspect로 나누는것이 좋다고 느낌.

In [ ]:
index_list = list(all_memo_data.columns[:])

In [ ]:
index_list

['index',
 'lesion_id',
 'car_id',
 'lesion_main_type',
 'lesion_sub_type',
 'lesion_detail_type',
 'lesion_created_at',
 'lesion_closed_at',
 'memo_id',
 'memo',
 'memo_created_at',
 'memo_updated_at',
 'inspect_id',
 'inspect_at',
 'inspect_type',
 'price',
 'description']

In [ ]:
index_list[:6]

['index',
 'lesion_id',
 'car_id',
 'lesion_main_type',
 'lesion_sub_type',
 'lesion_detail_type']

In [ ]:
index_list[12:]

['inspect_id', 'inspect_at', 'inspect_type', 'price', 'description']

In [ ]:
inspect_index = index_list[:6]
inspect_index +=(index_list[12:])

inspect_index # 필요한 열들만 추출

['index',
 'lesion_id',
 'car_id',
 'lesion_main_type',
 'lesion_sub_type',
 'lesion_detail_type',
 'inspect_id',
 'inspect_at',
 'inspect_type',
 'price',
 'description']

### **memo_data / inspect_data 나누기**

In [ ]:
def remove_space_sp(memo):
    temp_sents = memo.split('\n')
    temp_sent = []
    for n, i in enumerate(temp_sents):
        if i == '':
            return ''
        if i[-1] == ' ':
            temp_sent.append(i[:-1])
            continue
        temp_sent.append(i)

    my_sents = '\n'.join(temp_sent)

    return my_sents

In [ ]:
memo_data = all_memo_data[index_list[:12]]
inspect_data = all_memo_data[inspect_index]

In [ ]:
# memo_data.head()

In [ ]:
# memo_data['memo'] = memo_data['memo'].astype(str)

In [ ]:
# for n,i in enumerate(memo_data['memo']):
#     try:
#         memo_data['memo'][n] = remove_space_sp(i)
#     except:
#         print(n, i)

In [ ]:
memo_data = memo_data.drop(['index'], axis=1, inplace=False)
# memo_data.head()

In [ ]:
len(memo_data)

99180

memo_data 중복 행 제거

In [ ]:
memo_list = []
memo_drop_list = []
for i in range(len(memo_data)):
    if memo_data['memo'][i] in memo_list:
        memo_drop_list.append(i)
    else:
        memo_list.append(memo_data['memo'][i])

memo_data = memo_data.drop(index=memo_drop_list, axis=0, inplace=False)
# memo_data.head()

In [ ]:
len(memo_data)

42641

In [ ]:
memo_data['lesion_id'].nunique()

9452

In [ ]:
len(memo_data)/memo_data['lesion_id'].nunique()

4.511320355480322

한 개의 lesion_id당 약 5.4개의 memo가 기록된다는것을 알 수 있습니다.

index를 reset하여 다시 순서정렬합니다.

In [ ]:
memo_data = memo_data.reset_index(drop=True)
# memo_data.head()

In [ ]:
memo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42641 entries, 0 to 42640
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   lesion_id           42641 non-null  int64 
 1   car_id              42641 non-null  int64 
 2   lesion_main_type    42641 non-null  object
 3   lesion_sub_type     42641 non-null  object
 4   lesion_detail_type  42641 non-null  object
 5   lesion_created_at   42641 non-null  object
 6   lesion_closed_at    42418 non-null  object
 7   memo_id             42641 non-null  int64 
 8   memo                42640 non-null  object
 9   memo_created_at     42641 non-null  object
 10  memo_updated_at     42641 non-null  object
dtypes: int64(3), object(8)
memory usage: 3.6+ MB


In [ ]:
nav_drop_memo_idx = []

for i in range(len(memo_data)):
    if memo_data['lesion_sub_type'][i] != '내비게이션':
        nav_drop_memo_idx.append(i)

nav_memo = memo_data.drop(index=nav_drop_memo_idx, axis=0, inplace=False)

nav_memo = nav_memo.reset_index(drop=True)
print(len(nav_memo))

6675


In [ ]:
nav_memo = nav_memo[['lesion_id', 'lesion_sub_type', 'lesion_detail_type', 'memo']]

In [ ]:
nav_summemo = []
for i in list(nav_memo['lesion_id'].unique()):
    temp_summemo = []
    for j in range(len(nav_memo)):
        if nav_memo['lesion_id'][j] == i:
            temp_summemo.append(str(nav_memo['memo'][j]))
    temp_summemo = '\n'.join(temp_summemo)
    nav_summemo.append(temp_summemo)

print(len(nav_summemo))

1461


In [ ]:
nav_memo = nav_memo.drop_duplicates(['lesion_id'])
nav_memo

---

In [ ]:
nav_memo['memo'] = nav_summemo
nav_memo = nav_memo.reset_index(drop=True)
nav_memo

In [ ]:
# for i in range(len(nav_memo)):
#     print(nav_memo['memo'][i])
#     print('==='*10)

In [ ]:
# nav_memo['memo'][15]

In [ ]:
# nav_memo['memo'][15].split('\n')

In [ ]:
a = nav_memo['memo'][15].split('\n')
for i in a:
    if '전원가능여부' in i:
        print(i[3:11])
        print(i[13:14])

전원가능여부 :
X


In [ ]:
def check_lost(memo):
    result = ''
    memo = memo.split('\n')
    for i in memo:
        if '분실여부' in i:
            if 'X' in i or 'x' in i:
                result = '0'
                break
            elif 'O' in i or 'o' in i or '케이블분실' in i:
                result = '1'
                break
            elif '-' in i or '()' in i or '( )' in i or 'O/X' in i:
                result = '-'
                break
            else:
                result = "Can't find anything"
                break
    if result == '':
        return '-'
    else:
        return result

In [ ]:
def check_power(memo):
    result = ''
    memo = memo.split('\n')
    for i in memo:
        if '전원가능여부' in i:
            if 'X' in i or 'x' in i:
                result = '0'
                break
            elif 'O' in i or 'o' in i or '0' in i:
                result = '1'
                break
            elif '-' in i or '()' in i or '( )' in i or 'O/X' in i or '확인필요' in i or '확인 필요' in i or '?' in i:
                result = '-'
                break
            else:
                result = "Can't find anything"
                break
    if result == '':
        return '-'
    else:
        return result

In [ ]:
def check_SD(memo):
    result = ''
    memo = memo.split('\n')
    for i in memo:
        if '내비SD카드 업데이트 요청여부' in i:
            if 'X' in i or 'x' in i or '전원안들어옴' in i:
                result = '0'
                break
            elif 'O' in i or 'o' in i:
                result = '1'
                break
            elif '-' in i or '()' in i or '( )' in i or 'O/X' in i:
                result = '-'
                break
            else:
                result = "Can't find anything"
                break
    if result == '':
        return '-'
    else:
        return result

In [ ]:
print(check_lost(nav_memo['memo'][15]))
print(check_power(nav_memo['memo'][15]))
print(check_SD(nav_memo['memo'][15]))

0
0
0


In [ ]:
ck_lost = []
ck_power = []
ck_sd = []

for i in range(len(nav_memo)):
    ck_lost.append(check_lost(nav_memo['memo'][i]))
    ck_power.append(check_power(nav_memo['memo'][i]))
    ck_sd.append(check_SD(nav_memo['memo'][i]))

In [ ]:
wrong_list1 = []
none_list1 = []

wrong_list2 = []
none_list2 = []

wrong_list3 = []
none_list3 = []
for i in range(len(ck_lost)):
    if ck_lost[i] == None:
        none_list1.append(i)
    if ck_lost[i] == "Can't find anything":
        wrong_list1.append(i)

for i in range(len(ck_power)):
    if ck_power[i] == "Can't find anything":
        wrong_list2.append(i)
    if ck_power[i] == None:
        none_list2.append(i)

for i in range(len(ck_sd)):
    if ck_sd[i] == "Can't find anything":
        wrong_list3.append(i)
    if ck_sd[i] == None:
        none_list3.append(i)
    

print(len(none_list1) + len(wrong_list1))
print(len(none_list2) + len(wrong_list2))
print(len(none_list3) + len(wrong_list3))

0
0
0


In [ ]:
print(len(none_list1))
print(len(wrong_list1))
print(len(none_list2))
print(len(wrong_list2))
print(len(none_list3))
print(len(wrong_list3))

0
0
0
0
0
0


In [ ]:
row_nav_memo = nav_memo.copy()
nav_memo['분실여부'] = ck_lost
nav_memo['전원가능여부'] = ck_power
nav_memo['내비SD카드 업데이트 요청여부'] = ck_sd

In [ ]:
# for i in range(len(nav_memo)):
#     print(nav_memo['memo'][i])
#     print('==='*10)

In [ ]:
# print(nav_memo['memo'][29])

In [ ]:
def split_to_newline(sentence, who):
    # 해당 단어(ㅁ, -, ㄴ)가 맨 처음에 존재하거나 or 맨 마지막 값이 '\n'이면 -> 고려 대상 x
    if sentence[sentence.find(who)-1] == '\n' or sentence.find(who) == 0:
        return sentence

    # 그렇지 않을 경우는 앞에 '\n' 추가해줘야 함
    sentence = sentence.split(who) # ㅁ, -, ㄴ 기준으로 문장 나눈 뒤
    nl_who = '\n' + who # 엔터 추가
    sentence = nl_who.join(sentence) # 엔터 추가 후 합치기
    return sentence

In [ ]:
def cc_trans(sentence):
    temp_sent = sentence.split('\n')
    for i, temp_sent_str in enumerate(temp_sent):
        # (CC) 뒤에 엔터가 없을 경우 처리해주기 위해서
        if '(CC)' in temp_sent_str:
            temp_sent_str = split_to_newline(temp_sent_str, 'ㅁ')
            temp_sent_str = split_to_newline(temp_sent_str, '-')
            temp_sent_str = split_to_newline(temp_sent_str, 'ㄴ')
            temp_sent[i] = temp_sent_str
            break

    return '\n'.join(temp_sent)

In [ ]:
# # 원본 -> (CC) 뒤 내용 줄바꿈 안 됨
# print(nav_memo['memo'][29])

In [ ]:
# # 수정 후 -> (CC) 뒤에 줄 바꿈 잘 됨
# print(cc_trans(nav_memo['memo'][29]))

In [ ]:
# 추가
index = [29, 167, 215, 326, 418, 429, 691, 804, 1062, 1238]

for idx in index:
    nav_memo['memo'][index] = cc_trans(nav_memo['memo'][idx])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# nav_memo['memo'][29] = cc_trans(nav_memo['memo'][29])

In [ ]:
# # 잘 됐는지 확인
# print((nav_memo['memo'][29]))

In [ ]:
lost_list = []
lost_count = 0
for i in range(len(ck_lost)):
    if ck_lost[i] == '1': # 분실 한 경우
        lost_count +=1
        lost_list.append(i)
print(lost_count)

13


In [ ]:
power_list = []
power_count = 0
for i in range(len(ck_power)):
    if ck_power[i] == '0': # 안 켜지는 경우 
        power_count += 1
        power_list.append(i)
print(power_count)

264


In [ ]:
sd_list = []
sd_count = 0
for i in range(len(ck_sd)):
    if ck_sd[i] == '1': # sd카드 불량 문제
        sd_count += 1
        sd_list.append(i)
print(sd_count)

123


In [ ]:
# for i in lost_list:
#     print(nav_memo['memo'][i])
#     print('===='*10)

In [ ]:
print(len(nav_memo['memo']))

1461


In [ ]:
nav_memo['memo'].nunique()

1452

In [ ]:
count = 0
as_is_in = []
for i in range(len(nav_memo['memo'])):
    if '증상 상세내용' in nav_memo['memo'][i]:
        count += 1
        as_is_in.append(i)

print(count)

1419


> 정연님: 1426개  // 35개

In [ ]:
count = 0
as_is_not_in = []
for i in range(len(nav_memo['memo'])):
    if '증상 상세내용' not in nav_memo['memo'][i]:
        count += 1
        as_is_not_in.append(i)

print(count)

42


In [ ]:
as_is_in_memo = []

for i in as_is_in:
    temp_sent = nav_memo['memo'][i].split('\n')
    for n, j in enumerate(temp_sent):
        if '증상 상세내용' in j:
            as_is_in_memo.append('\n'.join(temp_sent[n:])) # '증상 상세내용' 부터 맨 아래까지 다 저장 (띄어쓰기 구분으로 분리했으므로 다시 띄어쓰기 기준으로 합치기)
            # print(as_is_in_memo)
            break

In [ ]:
# print(as_is_in_memo[0])

In [ ]:
# for i in as_is_in_memo:
#     print(i)
#     print('-'*50)

In [ ]:
# as_is_in_memo[:2]

In [ ]:
a = 'ㅁ문의내용 '

if '문의내용' in a:
  print('yes')

yes


In [ ]:
inq = []
inq_index = []
nothing = []

# 1324개
for i in as_is_in:
    sentence = nav_memo['memo'][i]
    temp_sent = sentence.split('\n')

    start_idx = 10000
    for n, j in enumerate(temp_sent): # 한 줄 한 줄 살펴보기
        if 'ㅁ문의내용' in j or 'ㅁ 문의내용' in j or ')문의내용' in j or 'ㅁ문의 내용' in j or 'ㅁ 문의사항' in j or 'ㅁ문의사항' in j:
            start_idx = n # '문의내용'이 있는 데이터의 index
            # inq_index.append(index)

        if n > start_idx and 'ㅁ' in j: # 이후 줄에 'ㅁ'이 있다면
            inq_sep = '\n'.join(temp_sent[start_idx: n]) # '문의내용'부터 'ㅁ' 전까지 자르기
            inq.append(inq_sep)
            # print(index)
            inq_index.append(i)
            break


        if '◎문의 내용' in j:
            start_idx = n 
        if n > start_idx and '◎' in j:
            inq_sep = '\n'.join(temp_sent[start_idx: n])
            inq.append(inq_sep)
            inq_index.append(i)
            break


        if 'ㅁ문의구분' in j:
            start_idx = n 
        if n > start_idx and 'ㅁ' in j:
            inq_sep = '\n'.join(temp_sent[start_idx: n])
            inq.append(inq_sep)
            inq_index.append(i)
            break

        
        if '문의내용' in j:
            start_idx = n 
        if n > start_idx and ('확' in j or 'ㅁ' in j):
            inq_sep = '\n'.join(temp_sent[start_idx: n])
            inq.append(inq_sep)
            inq_index.append(i)
            break


        # if '증상 상세내용' in j:
        #     start_idx = n 
        # if n > start_idx and 'ㅁ' in j:
        #     inq_sep = '\n'.join(temp_sent[start_idx: n])
        #     inq.append(inq_sep)
        #     inq_index.append(i)
        #     break

In [ ]:
len(inq), len(inq_index)

(1352, 1352)

In [ ]:
# nothing_index = list(nav_memo.index)
nothing_index = list(as_is_in)
# after_index
for x in inq_index:
  nothing_index.remove(x)

print(nothing_index)
print(len(nothing_index))

[42, 63, 105, 109, 122, 126, 185, 202, 248, 257, 280, 351, 403, 405, 412, 431, 449, 452, 499, 506, 524, 561, 567, 584, 634, 665, 669, 673, 689, 698, 710, 714, 774, 814, 828, 852, 905, 945, 1023, 1072, 1080, 1096, 1099, 1118, 1134, 1138, 1196, 1197, 1198, 1203, 1211, 1218, 1220, 1224, 1226, 1260, 1277, 1284, 1320, 1344, 1359, 1387, 1392, 1401, 1402, 1432, 1448]
67


In [ ]:
nothing_index[5:10]

[126, 185, 202, 248, 257]

In [ ]:
print('원본 길이: \t', len(as_is_in_memo))
print('-'*30)
print('문의내용 포함: \t', len(inq))
print('문의내용 미포함: \t', len(nothing_index))
print('합: \t\t', len(inq) + len(nothing_index))

원본 길이: 	 1419
------------------------------
문의내용 포함: 	 1352
문의내용 미포함: 	 67
합: 		 1419


In [ ]:
for i in nothing_index:
    if 'ㅁ문의내용' in nav_memo['memo'][i]:
      print(i)

> - ~ㅁ이 없는 문의내용~
> - ~ㅁ문의 내용~
> - ~₩)문의내용~

> - ~ㅁ문의구분~
> - ~◎문의 내용~
>> ~얘는 뒤에도 ◎ 인듯~

>> ~ㅁ이 없는 문의내용이 많아 그냥 문의내용을 포함 시킬까 하더라도, 뒤에 ㅁ확인내용인 경우, 그냥 확인내용인 경우 등의 경우의 수가 다양한 것 같다.~

In [ ]:
# for i in nothing_index:
#   print(nav_memo['memo'][i])
#   print('-'*50)

In [ ]:
# test
a = []
a_index = []

for index, sentence in enumerate(nav_memo['memo'][idx]):

    temp = sentence.split('\n')

    start_idx = 10000

    for i, line in enumerate(temp):
      if 'ㅁ문의내용' in line:
        print('발견: ', i)
        start_idx = i
      if i > start_idx and 'ㅁ' in line:
          print('다음 줄 발견 - 첫 인덱스: ', start_idx)
          print('다음 줄 발견 - 끝 인덱스: ', i)
          a_sep = '\n'.join(temp[start_idx: i])
          a.append(a_sep)
          a_index.append(index)
          break

In [ ]:
# for i in inq:
#     print(i)
#     print('-'*50)

In [ ]:
len(inq)

1352

In [ ]:
jck = []
for i in inq:
    i = i.split('\n')
    for n, j in enumerate(i):
        if '문의내용' in j:
            new_i = i[n+1:]
            break
    for k in new_i:
        if '문의내용' in k:
            print(k)
            print('-'*50)

'문의내용'이 두 번 반복되는 경우는 없다는 것을 확인.

In [ ]:
def remove_as(sentence):
    while True:
        s_loc = sentence.find('ㄴ')
        u_loc = sentence.find('-')
        st_loc = sentence.find('*')
        st2_loc = sentence.find('＊')

        # 없으면 그대로
        if s_loc == -1 and u_loc == -1 and st_loc == -1 and st2_loc == -1:
            break
        
        # 있으면 제외
        if s_loc != -1:
            sentence = sentence[:s_loc] + sentence[s_loc+1:]
        if u_loc != -1:
            sentence = sentence[:u_loc] + sentence[u_loc+1:]
        if st_loc != -1:
            sentence = sentence[:st_loc] + sentence[st_loc+1:]
        if st2_loc != -1:
            sentence = sentence[:st2_loc] + sentence[st2_loc+1:]

    return sentence


In [ ]:
cnt1 = 0
cnt2 = 0
cnt3 = 0
cnt4 = 0
cnt5 = 0

for line in inq:
  if '문의내용' in line:
    cnt1 += 1

  if '문의 내용' in line:
    cnt2 += 1

  if '문의사항' in line:
    cnt3 += 1

  if '문의구분' in line:
    cnt4 += 1

print('문의내용: \t', cnt1)
print('문의 내용: \t', cnt2)
print('문의사항: \t', cnt3)
print('문의구분: \t', cnt4)
print('-'*40)
print('전체: \t\t', cnt1+cnt2+cnt3+cnt4)

문의내용: 	 1221
문의 내용: 	 4
문의사항: 	 123
문의구분: 	 4
----------------------------------------
전체: 		 1352


In [ ]:
# for line in inq[:5]:
#   print(line)
#   print('-'*30)

In [ ]:
inq_ans = []
cnt=0

for line in inq:
    line = line.split('\n')
    for i, sen in enumerate(line):
        if '문의내용' in sen or '문의 내용' in sen or '문의사항' in sen or '문의구분' in sen:
            new = line[i+1:]
            new = '\n'.join(new)
            new = remove_as(new)
            inq_ans.append(new)
            break

print(len(inq_ans))

1352


In [ ]:
# inq_ans = []

# for i in inq:
#     i = i.split('\n')
#     for n, j in enumerate(i):
#         if '문의내용' in j:
#             new_i = i[n+1:] # '문의내용' 뒤부터
#             new_i = '\n'.join(new_i)
#             new_i = remove_as(new_i)
#             break
    
#     inq_ans.append(new_i)

In [ ]:
# # my_i = '\n'.join(inq_ans)
# for i in inq_ans:
#     print(i)
#     print('-'*50)

In [ ]:
import re

s = ' I       Hey    hi  '
s = re.sub(' +', ' ', s)
print(s)

 I Hey hi 


In [ ]:
import re

def remove_space(sentence):
    while True:
        sentence = re.sub(' +', ' ', sentence) # 다중 공백 제거
        if sentence[0] == ' ': # case 1. 첫 번째 글자 공백
            sentence = sentence[1:]
        elif sentence[-1] == '\n' or sentence[-1] == ' ': # case 2. 마지막 글자 '\n'이거나 공백
            sentence = sentence[:-1]
        else:
            return(sentence)

In [ ]:
cnt = 0

for sen in inq_ans:
  if len(sen) > 0:
    cnt += 1

print(cnt)

1028


In [ ]:
my_q = []
for sen in inq_ans:
    if sen == '': # 빈 문자열이라면 for문으로 다시 돌아감
        continue
    my_q.append(remove_space(sen)) # 빈 문자열 아니라면 공백 제거

In [ ]:
print(len(my_q)) # 같음 !!

1028


In [ ]:
# for i in my_q:
#     print(i)
#     print('='*50)

In [ ]:
len(my_q)

1028

### **일단 keep 이요 ....**

In [ ]:
# sample = my_q[:10]

In [ ]:
# sample

In [ ]:
# nav_first = []
# while True:
#     temp = str(input('첫마디set:   [끝내시려면 "-1"]\n'))
#     if temp == '-1':
#         break
#     temp = temp+'\n'
#     nav_first.append(temp)

In [ ]:
# nav_first

In [ ]:
# import random

# def create_answer(first_sent, sentence_name): # sentence_name == 구분하기 위한 str 문구
#     temp_list = []
#     while True:
#         temp = str(input(f'{sentence_name}:  [끝내시려면 "-1"]\n'))
#         if temp == '-1':
#             break
        
#         rannum = random.randrange(1, len(first_sent))
#         temp = first_sent[rannum] + temp
#         temp_list.append(temp)

#     return temp_list

In [ ]:
# answer_set1 = []
# answer_set1 = create_answer(nav_first, "쿠폰 지급 관련")

In [ ]:
# answer_set1

In [ ]:
# for i in answer_set1:
#     print(i)
#     print('==='*10)

In [ ]:
# nav_first

In [ ]:
# for n, i in enumerate(nav_first):
#     temp = i + '\n' # 엔터 추가?
#     nav_first[n] = temp

In [ ]:
# nav_first

In [ ]:
# import random

# new_q = []
# for i in sample_answer:
#     rannum = random.randrange(1, len(nav_first))
#     temp_sent = nav_first[rannum] + i
#     new_q.append(temp_sent)

In [ ]:
# for i in new_q:
#     print(i)
#     print('==='*10)

In [ ]:
# cd /content/drive/MyDrive/AIFFELTHON/our_module

In [ ]:
# import create_navi_answer as cn

In [ ]:
# cn = cn.create_navi_answer(0, 11)

In [ ]:
# my_fir_sent_1 = cn.first_sent()

In [ ]:
# navi_set_1 = cn.create_answer(my_fir_sent, "가이드 관련")

In [ ]:
# navi_set_2 = cn.create_answer(my_fir_sent, "쿠폰 지급 관련")

In [ ]:
# for i in navi_set_1:
#     print(i)
#     print('---'*10)

In [ ]:
# for i in my_q[1:11]:
#     print(i)
#     label_1 = str(input())
#     print("")

----
### **Trying 노드 6**

In [ ]:
# my_q

In [ ]:
# for idx, q in enumerate(my_q[:10]):
  # print(idx, q)

In [ ]:
komoran = Komoran() # 형태소 분석 시작

def to_token_ko(my_q):
  '''
  make sentence to subword sentence
  '''
  morph_q = []
  for q in my_q:
      morph_out = komoran.morphs(q.replace(';','').replace('\n', '. ')) # 특수문자 -> 공백 변환 후 각 형태소 별로 분리
      morph_q.append(' '.join(morph_out))

  return morph_q

In [ ]:
# 더 안 좋아진 것 같음...
# to_token_ko(my_q[:10])


#### **🔆 자동차 사전 준비하기**

In [ ]:
# load car dictionary
def load_car_dic():
    car_dic_file = open('/content/drive/MyDrive/data/socar_memo/carDic.txt', 'r')
    car_dic = set(car_dic_file.read().split('\n'))
    return car_dic

car_dic = load_car_dic()
print(len(car_dic)) #1433

1433


In [ ]:
# '\n'.join(my_q)

In [ ]:
PATH = '/content/drive/MyDrive/AIFFELTHON/정연'

#sentenctePiece를 학습하기 위해서는 별도의 text 파일이 필요함
#따라서 이를 만드는 부분
# 뉴스 기사 데이터(KCC150_100k.txt)는 토큰화를 잘 시키기 위해 가져온 정제된 데이터
with open(PATH + '/car_info_memo.txt', 'w') as w, open(PATH + '/KCC150_100k.txt', 'r') as corpus:
    w.write('\n'.join(my_q))
    for idx, snt in enumerate(corpus.readlines()):
        w.write(snt)

In [ ]:
#SentencePiece Train & load
spm.SentencePieceTrainer.Train(f'--input={PATH}/car_info_memo.txt --model_prefix={PATH}/description --vocab_size=6000')
sp = spm.SentencePieceProcessor()
vocab_file = PATH + '/description.model'
sp.load(vocab_file)

True

In [ ]:
def to_token_sp(my_q):
  '''
  make sentence to subword sentence
  '''
  morph_q = []
  for q in my_q:
      sp_out = sp.encode_as_pieces(q.replace(';','').replace('\n', '. ')) # 특수문자 -> 공백 변환 후 각 형태소 별로 분리
      morph_q.append(' '.join(token.replace('▁','') for token in sp_out))

  return morph_q

In [ ]:
# my_q[:10]

In [ ]:
# to_token_ko(my_q[:10])

In [ ]:
my_q1 = to_token_ko(my_q)

In [ ]:
# to_token_sp(my_q[:10])

In [ ]:
after_my_q = to_token_sp(my_q)

In [ ]:
# for line in after_my_q[:3]:
  # print(line.split())

In [ ]:
# 토큰화한 데이터를 이용해 단어 벡터 학습시키기
# Word2Vec train data processing
sentences = [line.split() for line in after_my_q]

# FastText 훈련시키기
# sentences에 데이터를 전처리 해두었다고 가정합니다.
model_f = gensim.models.fasttext.FastText(size=300, window=5, min_count=1)
model_f.build_vocab(sentences)
model_f.train(sentences, total_examples=len(sentences), epochs=10)

In [ ]:
# Word2Vec train part
# vector_size -> size
# epochs -> iter

model_w = gensim.models.word2vec.Word2Vec(sentences, size=300, window=5, min_count=1, iter=10)

> 확실히 word2vec보단 fasttext가 더 토큰화 잘 함

In [ ]:
# check word most similarity
model_w.wv.most_similar('네비게이션', topn=10)

[('.', 0.9999449849128723),
 ('/', 0.9999434351921082),
 ('지', 0.9999417066574097),
 (',', 0.999938428401947),
 ('에', 0.9999361634254456),
 ('을', 0.9999359846115112),
 ('차량', 0.9999352693557739),
 ('주', 0.9999351501464844),
 ('는데', 0.9999347925186157),
 ('도', 0.999934732913971)]

In [ ]:
# check word most similarity
model_f.wv.most_similar('네비게이션', topn=10)

[('내비게이션', 0.9999831914901733),
 ('내비', 0.9999620318412781),
 ('.', 0.9999578595161438),
 ('에', 0.9999555349349976),
 ('는데', 0.9999549388885498),
 ('으로', 0.9999536871910095),
 ('/', 0.99995356798172),
 ('있', 0.999953031539917),
 ('사용', 0.999951183795929),
 ('이용', 0.999950110912323)]

In [ ]:
# check word most similarity
model_w.wv.most_similar('내비게이션', topn=10)

[('.', 0.999950647354126),
 (',', 0.9999493360519409),
 ('도', 0.9999470710754395),
 ('는데', 0.9999450445175171),
 ('지', 0.9999428391456604),
 ('안내', 0.9999427795410156),
 ('전', 0.9999421834945679),
 ('/', 0.9999418258666992),
 ('을', 0.9999417066574097),
 ('주', 0.9999412298202515)]

In [ ]:
# check word most similarity
model_f.wv.most_similar('내비게이션', topn=10)

[('네비게이션', 0.9999831914901733),
 ('내비', 0.9999616742134094),
 ('.', 0.9999518394470215),
 ('다', 0.999945878982544),
 ('에', 0.9999445676803589),
 ('/', 0.999943733215332),
 ('있', 0.9999435544013977),
 ('이용', 0.9999434351921082),
 (',', 0.9999430179595947),
 ('는데', 0.9999426007270813)]

In [ ]:
# check word most similarity
model_w.wv.most_similar('내비', topn=10)

[('/', 0.999954879283905),
 ('.', 0.999954342842102),
 (',', 0.9999514818191528),
 ('함', 0.9999495148658752),
 ('안내', 0.9999490976333618),
 ('을', 0.9999479055404663),
 ('차량', 0.9999477863311768),
 ('에', 0.9999469518661499),
 ('도', 0.9999468922615051),
 ('해', 0.9999462366104126)]

In [ ]:
# check word most similarity
model_f.wv.most_similar('내비', topn=10)

[('네비게이션', 0.9999620914459229),
 ('내비게이션', 0.9999616742134094),
 ('.', 0.9999516606330872),
 ('는데', 0.9999483823776245),
 ('고', 0.9999480247497559),
 ('했으나', 0.9999459385871887),
 ('에', 0.9999450445175171),
 ('다', 0.9999449253082275),
 ('안내', 0.9999445080757141),
 ('하여', 0.9999443292617798)]

In [ ]:
# 단어 벡터에서 유사도가 평균값 이상인 경우만 선택하기
def exclusion(word_list):
    '''
    most_similar의 단어를 평균 이상만 추림
    '''
    count = sum([score for _, score in word_list])
    avg = count / len(word_list)
  
    up_word = []
    for w, s in word_list:
        if s > avg:
            up_word.append(w)
  
    return up_word

In [ ]:
navi = model_f.wv.most_similar('내비게이션', topn=5)
navi_word = exclusion(navi)
navi = model_f.wv.most_similar('내비', topn=5)
navi_word.extend(exclusion(navi))
navi = model_f.wv.most_similar('네비', topn=5)
navi_word.extend(exclusion(navi))

navi_word.append('내비게이션')
navi_word.append('네비게이션')
navi_word.append('내비')
navi_word.append('네비')


navi_word = set(navi_word) # 중복 제거

navi_word

{'내비', '내비게이션', '네비', '네비게이션'}

In [ ]:
# 매핑 테이블 작성
mapping_table = {'내비게이션':'네비게이션', '내비':'네비게이션', '네비':'네비게이션'}

print(mapping_table)

{'내비게이션': '네비게이션', '내비': '네비게이션', '네비': '네비게이션'}


In [ ]:
def to_token_sp(my_q):
  '''
  make sentence to subword sentence
  '''
  morph_q = []
  for q in my_q:
      sp_out = sp.encode_as_pieces(q.replace(';','').replace('\n', '. ')) # 특수문자 -> 공백 변환 후 각 형태소 별로 분리
      morph_q.append(' '.join(token.replace('▁','') for token in sp_out))

  return morph_q

In [ ]:
# 카테고리 내 공백 들어간 경우 고려
def noun_phrase(words):
    result = []

    for index, token in enumerate(words):
        if token in car_dic:
            #명사구 check # 공백이 있으면 그 자체로 넣기
            if ''.join(words[index : index +2]) in car_dic:
                result.append(''.join(words[index : index +2])) # ['a', 'b'] -> 'ab'
            #명사구 X
            result.append(token)
    return result


# 어절 단위로 분절한 결과 -> 카테고리 찾기
def find_category(q):
    # word base
    line = q.replace(';','').replace('\n', '. ')
    car_token = noun_phrase(line.upper().split()) # 특수문자 -> 공백으로 변환 해서 noun_phrase에 넣음
    # subword base
    car_token.extend(noun_phrase(q.upper().split()))

    return set(car_token) # 중복 제거 해서 !

In [ ]:
subword_list = []

for q in my_q:
    words = find_category(q) # find word in car_dic

    if len(words) >= 1:
        subword_list.extend(words)

subword_list = [subword for subword in subword_list if subword != '']
subword_counter = Counter(subword_list)
subword_sort = sorted(subword_counter.items(), key=lambda x:x[1], reverse=True)
subword_top20 = [(key, value) for key, value in subword_sort[:20]]

print(subword_top20)


# for _, row in memo_data.iterrows():
#     if row['inspect_type'] == '일반수리':
#         car_words = find_category(row) #find word in car_dic
    
#         if len(car_words) >= 1:  #car_words more than one
#             subword_list.extend(car_words)

# subword_list = [subword for subword in subword_list if subword != '']
# subword_counter = Counter(subword_list) #car_dic에 포함되는 단어 + category에 포함되는 단어
# subword_sort = sorted(subword_counter.items(), key=lambda x:x[1], reverse=True) # value 기준으로 내림차순 정렬
# subword_top20 = [(key, value) for key, value in subword_sort[:20]]

# print(subword_top20)

[('전원', 10), ('불량', 4), ('전조등', 4), ('제동등', 1), ('입력', 1), ('연료', 1), ('센서', 1), ('시트', 1)]


---

#### **뉴스 정제 기사 데이터 제외**

In [ ]:
PATH = '/content/drive/MyDrive/AIFFELTHON/정연'

#sentenctePiece를 학습하기 위해서는 별도의 text 파일이 필요함
#따라서 이를 만드는 부분
# 뉴스 기사 데이터(KCC150_100k.txt)는 토큰화를 잘 시키기 위해 가져온 정제된 데이터
with open(PATH + '/car_info_memo_original.txt', 'w') as w:
    w.write('\n'.join(my_q))

In [ ]:
def load_car_dic_original():
    car_dic_file = open(PATH + '/car_info_memo_original.txt', 'r')
    car_dic_original = set(car_dic_file.read().split('\n'))
    return car_dic_original

car_dic_original = load_car_dic_original()
print(len(car_dic_original)) #950

950


In [ ]:
#SentencePiece Train & load
spm.SentencePieceTrainer.Train(f'--input={PATH}/car_info_memo_original.txt --model_prefix={PATH}/description_original --vocab_size=1403')
sp = spm.SentencePieceProcessor()
vocab_file = PATH + '/description_original.model'
sp.load(vocab_file)

True

In [ ]:
# 토큰화한 데이터를 이용해 단어 벡터 학습시키기
# Word2Vec train data processing
sentences = [line.split() for line in after_my_q]

# FastText 훈련시키기
# sentences에 데이터를 전처리 해두었다고 가정합니다.
model_f = gensim.models.fasttext.FastText(size=300, window=5, min_count=1)
model_f.build_vocab(sentences)
model_f.train(sentences, total_examples=len(sentences), epochs=10)

In [ ]:
model_f.wv.most_similar('네비게이션', topn=10)

[('내비게이션', 0.9999831914901733),
 ('내비', 0.9999620318412781),
 ('.', 0.9999577403068542),
 ('에', 0.9999555945396423),
 ('는데', 0.9999549388885498),
 ('으로', 0.9999536275863647),
 ('/', 0.9999534487724304),
 ('있', 0.9999530911445618),
 ('사용', 0.9999513030052185),
 ('이용', 0.9999501705169678)]

In [ ]:
subword_list = []

for q in my_q:
    words = find_category(q) # find word in car_dic

    if len(words) >= 1:
        subword_list.extend(words)

subword_list = [subword for subword in subword_list if subword != '']
subword_counter = Counter(subword_list)
subword_sort = sorted(subword_counter.items(), key=lambda x:x[1], reverse=True)
subword_top20 = [(key, value) for key, value in subword_sort[:20]]

print(subword_top20)

[('전원', 10), ('불량', 4), ('전조등', 4), ('제동등', 1), ('입력', 1), ('연료', 1), ('센서', 1), ('시트', 1)]


### **0515**
> - 참조: https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=vangarang&logNo=221068634380

In [ ]:
# my_q[:10]

In [ ]:
from konlpy.tag import Twitter

text = """
도로와 철도 레일 위를 모두 달릴 수 있는 '레일버스'가 개발돼 벽지 노선 등에 투입될 것으로 보인다.
코레일은 "현재 25인승 버스를 개조해 만든 레일버스의 안전성 검증과 기술 추가 개발이 끝나면 3대를 제작해 이르면 2019년 말 강원 정선 지역의 정선선 등에 투입할 계획"이라고 6일 밝혔다.
레일버스는 버스 차량이 철도 레일 위에서도 달릴 수 있도록 일종의 보조 바퀴 역할을 하는 '가이드 휠'을 단 차량이다. 25~35인승 버스를 개조해 만들 계획이며 선로 위에서 시속 80㎞ 속도로 달릴 수 있다. 코레일 담당자는 "필요할 경우 차량 3대를 연결 운행하면 75~105명까지 탈 수 있을 것"이라고 말했다. 레일버스는 도로에서도 달릴 수 있어 기차역에 도착한 뒤 관광지, 버스 정류장까지 도로 주행이 가능하다. 도로를 달릴 때는 유압으로 가이드 휠을 들어 올리고 주행하는 방식이다.
코레일은 올해 초부터 레일버스 시험 차량을 제작해 대전의 시험 선로에서 운행하고 있다. 레일버스 개발에는 총 20억원이 들어갈 것으로 보이며, 차량 1대 제작 가격은 3억원 수준이다. 코레일 측은 "영국에서는 버스를 열차로 완전히 개조해 운행한 사례가 있으며, 일본은 2020년까지 레일버스와 형태가 비슷한 DMV(Dual Mode Vehicle)를 2020년까지 상용화할 계획"이라고 밝혔다.
코레일은 "레일버스가 투입되면 벽지 노선 적자는 줄이면서도 운행 횟수는 늘려나갈 수 있다"고 밝혔다. 예를 들여 정선선에 레일버스를 투입하면 정선선에서 발생하는 연간 적자를 18억원에서 13억원 수준으로 5억원 줄이면서 열차 운행 횟수는 현재(관광 열차 편도 2회)의 6.5배 수준인 편도 13회까지 늘릴 수 있을 것으로 기대하고 있다. 레일버스를 투입하면 현재 운행 중인 새마을·무궁화호급 열차보다 운영비 등을 줄일 수 있기 때문이다. 코레일은 이용객이 적고 적자가 나는 다른 벽지 노선 등에도 레일버스를 투입할 수 있을 것으로 기대하고 있다.
"""

def keyword_extractor(tagger, text):
    tokens = tagger.phrases(text)
    tokens = [ token for token in tokens if len(token) > 1 ] # 한 글자인 단어는 제외
    count_dict = [(token, text.count(token)) for token in tokens ]
    ranked_words = sorted(count_dict, key=lambda x:x[1], reverse=True)[:10]
    return [ keyword for keyword, freq in ranked_words ]

if __name__ == '__main__':
    twit = Twitter()
    print(keyword_extractor(twit, text))

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['레일', '버스', '레일버스', '투입', '코레일', '운행', '도로', '차량', '열차', '정선']


In [ ]:
# 위 함수에 넣기 위해 list를 sting으로 변환
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + ". "
    return result.strip()

result = listToString(my_q)
result1 = listToString(my_q1)
result2 = listToString(after_my_q)
# print(result)

In [ ]:
if __name__ == '__main__':
    twit = Twitter()
    # print(keyword_extractor(twit, result))

In [ ]:
if __name__ == '__main__':
    twit = Twitter()
    # print(keyword_extractor(twit, result1))

In [ ]:
if __name__ == '__main__':
    twit = Twitter()
    # print(keyword_extractor(twit, result2))

> - 먼저 토큰화가 잘 되어야 키워드 추출이 잘 되지 않을까.
> - or 먼저 mapping table로 '내비', '내비게이션', '네비', '네비게이션' 등을 '네비게이션'으로 통일한 후 키워드 추출을 진행?

In [ ]:
navi_word

{'내비', '내비게이션', '네비', '네비게이션'}

In [ ]:
mapping_table

{'내비': '네비게이션', '내비게이션': '네비게이션', '네비': '네비게이션'}

In [ ]:
for key, value in mapping_table.items():
  print(key, value)

내비게이션 네비게이션
내비 네비게이션
네비 네비게이션


In [ ]:
mapping_table.keys()

dict_keys(['내비게이션', '내비', '네비'])

#### **🔆 토큰화 진행**
> 참조: https://wikidocs.net/92961

In [ ]:
# print(result[:122])
print('-'*50)
# print(result1[:122])
print('-'*50)
# print(result2[:122])
print('-'*50)

In [ ]:
new_sent = result.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
# print(new_sent[:50])

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent)

NotImplementedError: ignored

In [ ]:
# result

In [ ]:
# Py-Hanspell은 네이버 한글 맞춤법 검사기를 바탕으로 만들어진 패키지입니다.

from hanspell import spell_checker

spelled_sent = spell_checker.check(result[:500]) # 전체가 안 되네
hanspell_sent = spelled_sent.checked

# print(hanspell_sent)

In [ ]:
if __name__ == '__main__':
    twit = Twitter()
    # print(keyword_extractor(twit, hanspell_sent))

In [ ]:
# after_my_q (result2)
spelled_sent2 = spell_checker.check(result2[:500])
hanspell_sent2 = spelled_sent2.checked

twit = Twitter()
# print(keyword_extractor(twit, hanspell_sent2))

In [ ]:
# s = "네비게이션이 잘 안 됩니다."
# s = "네비게이션이 고장났어요."
# s = "내비 화면이 멈춰있네요"
s = "내비도 안 되고 차량 내부도 너무 더러워요"
# s = "하 안녕하세요.. 지금 내비를 누가 훔쳐갔는지 없네요. 그리고 차 내부도 너무 더럽습니다."

twit = Twitter()
print(keyword_extractor(twit, s))

NameError: ignored

In [ ]:
mapping_table

{'내비': '네비게이션', '내비게이션': '네비게이션', '네비': '네비게이션'}

In [ ]:
print(len(my_q))

1028


In [ ]:
cnt = 0

for idx, q in enumerate(my_q):
  
  keyword_list = keyword_extractor(twit, q)
  # print(keyword_list)

  for keyword in keyword_list:
      if keyword in mapping_table.keys():
          # print(idx, q)
          cnt += 1
          break
          # print(keyword)

In [ ]:
print(len(my_q), cnt)

1028 691


#### **Tring SOYNLP**

In [ ]:
from konlpy.tag import Okt
tokenizer = Okt()
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정'))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정']


In [ ]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

('2016-10-20.txt', <http.client.HTTPMessage at 0x7fc53a0112d0>)

In [ ]:
# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)

30091

In [ ]:
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.399 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [ ]:
word_score_table["반포한"].cohesion_forward

0.08838002913645132

In [ ]:
word_score_table["반포한강"].cohesion_forward # '반포한강'은 '반포한'보다 응집 확률이 높다.

0.19841268168224552

In [ ]:
# SOYNLP의 L tokenizer
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [ ]:
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.')

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [ ]:
twitter.add_dictionary('은경이', 'Noun')

In [ ]:
twitter.morphs('은경이는 사무실로 갔습니다.')

['은경이', '는', '사무실', '로', '갔습니다', '.']

---

In [ ]:
# 형태소 분리 잘 됨
after_token = twitter.morphs(hanspell_sent)

In [ ]:
twit = Twitter()
print(keyword_extractor(twit, after_token))

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


TypeError: ignored